# PDFPLUMBER

In [ ]:
import pdfplumber
import pandas as pd

def scrapPDF(ruta):
    tablas = []

    with pdfplumber.open(ruta) as pdf:
        for pagina in pdf.pages:
            for tabla in pagina.extract_tables():
                if tabla and len(tabla) > 1:
                    tablas.append(tabla)

    encabezados_objetivo = {"nombre", "tipo", "observaciones"}
    indice_inicio = None

    for i, tabla in enumerate(tablas):
        encabezados = [str(celda).strip().lower() for celda in tabla[0] if celda]
        if encabezados_objetivo.issubset(set(encabezados)):
            indice_inicio = i
            break

    if indice_inicio is not None:
        tablas_relevantes = tablas[indice_inicio:]
        dfs = []

        for tabla in tablas_relevantes:
            # Generar encabezados seguros y únicos
            colnames = []
            counts = {}
            for i, c in enumerate(tabla[0]):
                name = str(c).strip() if c else f"col_{i}"
                counts[name] = counts.get(name, 0) + 1
                if counts[name] > 1:
                    name = f"{name}_{counts[name]}"
                colnames.append(name)

            df_temp = pd.DataFrame(tabla[1:], columns=colnames)

            # Limpiar columnas duplicadas o vacías
            df_temp = df_temp.loc[:, ~df_temp.columns.duplicated()]
            df_temp.reset_index(drop=True, inplace=True)

            dfs.append(df_temp)

        # Concatenar todas las tablas (aunque tengan columnas distintas)
        df = pd.concat(dfs, ignore_index=True, join='outer')
        df = df.loc[:, ~df.columns.duplicated()]  # eliminar duplicadas finales

        # Limpieza opcional
        if "Nombre" in df.columns:
            df = df[df["Tipo"] == "Bibliografía"]
            df["Nombre"] = df["Nombre"].astype(str).str.replace("\\n", " ", regex=False)
            df = df[~df["Nombre"].str.contains(r"^https?://", na=False)]
    else:
        df = pd.DataFrame()

    return df


In [ ]:
directory = "Guias Docentes"

dfs = []

for file in os.listdir(directory):
    if file.endswith(".pdf"):  
        ruta = os.path.join(directory, file)
        df = scrapPDF(ruta)  

        if not df.empty:  
            df["archivo_origen"] = file  
            dfs.append(df)
df_final = pd.concat(dfs, ignore_index=True)


In [ ]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup

def scrapGoogleScholar(name):
    url = f"https://scholar.google.com/scholar?q={quote(name)}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    respuesta = requests.get(url, headers=headers)
    if respuesta.status_code != 200:
        raise Exception(f"Error al acceder a Google Scholar: {respuesta.status_code}")

    html = respuesta.text
    soup = BeautifulSoup(html, "html.parser")

    resultado = {}
    item = soup.select_one(".gs_r.gs_or.gs_scl")

    if item:
        titulo_elem = item.select_one(".gs_rt")
        autor_elem = item.select_one(".gs_a")
        link_elem = titulo_elem.find("a") if titulo_elem else None

        titulo = titulo_elem.get_text(strip=True) if titulo_elem else ""
        autores = autor_elem.get_text(strip=True) if autor_elem else ""
        enlace = link_elem["href"] if link_elem and link_elem.has_attr("href") else ""

        resultado["Título"] = titulo.replace("[PDF]", "").strip()
        resultado["Autores"] = autores.split("-")[0].strip()
        resultado["Enlace"] = enlace

    return resultado

dfs_resultado = []

for index, fila in df.iterrows():
    nombre = fila["Nombre"]

    dict_bibliografia = scrapGoogleScholar(nombre)
    df_temp = pd.DataFrame([dict_bibliografia])  
    dfs_resultado.append(df_temp)

df_final = pd.concat(dfs_resultado, ignore_index=True)
print(df_final.head())

# Conectar a PostGreSQL

CREATE TABLE asignaturas (
    id INT PRIMARY KEY
);

CREATE TABLE bibliografias (
    id INT PRIMARY KEY,
    asignaturaId INT,  -- primero se declara la columna
    nombre VARCHAR(100),
    autores VARCHAR(100),
    url VARCHAR(200),
    FOREIGN KEY (asignaturaId) REFERENCES asignaturas(id)  -- luego se define la FK
);


In [88]:
from sqlalchemy import create_engine

usuario = "userPSQL"
contraseña = "passPSQL"
host = "localhost"  
puerto = "5432"
base_datos = "postgres"

engine = create_engine(
    f"postgresql+psycopg2://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
)

df_final.to_sql('bibliografias', engine, if_exists="replace", index=False)


5

# Competencias